In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import time

# ======================================================
# CONFIGURATION
# ======================================================

plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["font.size"] = 11

x = sp.symbols("x")
n_vals = [4, 8, 12, 16, 20]

methodes = ["Laguerre", "Tchebychev", "Legendre", "Simpson", "Spline"]

# ======================================================
# QUADRATURES
# ======================================================

def gauss_legendre(f, a, b, n):
    nodes, w = np.polynomial.legendre.leggauss(n)
    xg = 0.5*(b-a)*nodes + 0.5*(a+b)
    return 0.5*(b-a)*np.dot(w, f(xg))

def gauss_laguerre(g, n):
    nodes, w = np.polynomial.laguerre.laggauss(n)
    return np.dot(w, g(nodes))

def gauss_tchebychev(f, n):
    k = np.arange(1, n+1)
    t = np.cos((2*k-1)*np.pi/(2*n))
    return np.pi/n * np.sum(f(t))

def simpson(f, a, b, n):
    if n % 2 != 0:
        n += 1
    h = (b-a)/n
    xg = np.linspace(a, b, n+1)
    y = f(xg)
    return (h/3)*(y[0]+y[-1]+4*np.sum(y[1:-1:2])+2*np.sum(y[2:-1:2]))

def spline_quadratique(f, a, b, n):
    xg = np.linspace(a, b, n+1)
    y = f(xg)
    if not np.isfinite(y).all():
        return np.nan
    h = xg[1]-xg[0]
    z = np.zeros(n+1)
    z[0]=(y[1]-y[0])/h
    for i in range(n):
        z[i+1]=2*(y[i+1]-y[i])/h-z[i]
    I=0
    for i in range(n):
        a_i=(z[i+1]-z[i])/(2*h)
        b_i=z[i]
        c_i=y[i]
        I+=a_i*h**3/3+b_i*h**2/2+c_i*h
    return I

# ======================================================
# ÉVALUATION
# ======================================================

def evaluer(calcul, n, I_exact):
    t0=time.perf_counter()
    try:
        I_num=calcul(n)
    except Exception:
        return np.nan, np.nan
    return abs(I_exact-I_num), time.perf_counter()-t0

# ======================================================
# TESTS (4 TYPES)
# ======================================================

tests = [

    # 1. Laguerre
    {
        "nom":"Laguerre",
        "f": sp.exp(-x)*(x**2+1),
        "a":0,
        "b":sp.oo
    },

    # 2. Tchebychev
    {
        "nom":"Tchebychev",
        "f": 1/sp.sqrt(1-x**2),
        "a":-1,
        "b":1
    },

    # 3. Combinaison
    {
        "nom":"Combinaison",
        "f": sp.exp(-x)/(sp.sqrt(1-x**2)+1e-6),
        "a":0,
        "b":1
    },

    # 4. Fonction quelconque
    {
        "nom":"Quelconque",
        "f": sp.sin(x)+x**2,
        "a":0,
        "b":2
    }
]

# ======================================================
# BOUCLE PRINCIPALE
# ======================================================

for test in tests:

    erreurs = {m:[] for m in methodes}
    temps   = {m:[] for m in methodes}

    f_sym = test["f"]
    a, b = test["a"], test["b"]
    I_exact = float(sp.integrate(f_sym, (x, a, b)))

    # ---- fonctions transformées ----

    # Laguerre : t = x/(1-x)
    def f_lag(x):
        t = x/(1-x)
        dt = 1/(1-x)**2
        return np.exp(t)*f_num(t)*dt

    # Tchebychev : affine [-1,1]
    def f_tch(x):
        t = 0.5*(b-a)*x + 0.5*(a+b)
        dt = 0.5*(b-a)
        return f_num(t)*dt*np.sqrt(1-x**2)

    f_num = sp.lambdify(x, f_sym, "numpy")

    for n in n_vals:
        for m in methodes:

            if m=="Laguerre":
                e,t=evaluer(lambda n:gauss_laguerre(f_lag,n),n,I_exact)

            elif m=="Tchebychev":
                e,t=evaluer(lambda n:gauss_tchebychev(f_tch,n),n,I_exact)

            elif m=="Legendre":
                e,t=evaluer(lambda n:gauss_legendre(f_num,a,b,n),n,I_exact)

            elif m=="Simpson":
                e,t=evaluer(lambda n:simpson(f_num,a,b,n),n,I_exact)

            elif m=="Spline":
                e,t=evaluer(lambda n:spline_quadratique(f_num,a,b,n),n,I_exact)

            erreurs[m].append(e)
            temps[m].append(t)

    # ==================================================
    # GRAPHE ERREUR
    # ==================================================

    plt.figure()
    for m in methodes:
        plt.plot(n_vals, erreurs[m], marker='o', label=m)

    plt.yscale("log")
    plt.title(f"Erreur numérique — {test['nom']}")
    plt.xlabel("n")
    plt.ylabel("Erreur (log)")
    plt.grid(True, which="both")
    plt.legend()
    plt.show()

    # ==================================================
    # GRAPHE TEMPS (BANDES)
    # ==================================================

    x_pos = np.arange(len(n_vals))
    width = 0.15

    plt.figure()
    for i,m in enumerate(methodes):
        plt.bar(x_pos+i*width, temps[m], width, label=m)

    plt.xticks(x_pos+2*width, [f"n={n}" for n in n_vals])
    plt.title(f"Temps d'exécution — {test['nom']}")
    plt.ylabel("Temps (s)")
    plt.grid(True, axis='y')
    plt.legend()
    plt.show()


In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import time

# ======================================================
# CONFIGURATION
# ======================================================

plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["font.size"] = 11

x = sp.symbols("x")
n_vals = [4, 8, 12, 16, 20]
methodes = ["Laguerre", "Tchebychev", "Legendre", "Simpson", "Spline"]

# ======================================================
# MÉTHODES NUMÉRIQUES
# ======================================================

def gauss_legendre(f, a, b, n):
    nodes, w = np.polynomial.legendre.leggauss(n)
    t = 0.5*(b-a)*nodes + 0.5*(a+b)
    return 0.5*(b-a)*np.dot(w, f(t))

def gauss_laguerre(g, n):
    nodes, w = np.polynomial.laguerre.laggauss(n)
    return np.dot(w, g(nodes))

def gauss_tchebychev(g, n):
    k = np.arange(1, n+1)
    xk = np.cos((2*k-1)*np.pi/(2*n))
    return np.pi/n * np.sum(g(xk))

def simpson(f, a, b, n):
    if n % 2 == 1:
        n += 1
    h = (b-a)/n
    xg = np.linspace(a, b, n+1)
    y = f(xg)
    return (h/3)*(y[0]+y[-1]+4*np.sum(y[1:-1:2])+2*np.sum(y[2:-1:2]))

def spline_quadratique(f, a, b, n):
    xg = np.linspace(a, b, n+1)
    y = f(xg)
    if not np.isfinite(y).all():
        return np.nan
    h = xg[1]-xg[0]
    z = np.zeros(n+1)
    z[0] = (y[1]-y[0])/h
    for i in range(n):
        z[i+1] = 2*(y[i+1]-y[i])/h - z[i]
    I = 0
    for i in range(n):
        ai = (z[i+1]-z[i])/(2*h)
        bi = z[i]
        ci = y[i]
        I += ai*h**3/3 + bi*h**2/2 + ci*h
    return I

# ======================================================
# OUTIL D'ÉVALUATION
# ======================================================

def evaluer(calcul, n, I_exact):
    t0 = time.perf_counter()
    try:
        I_num = calcul(n)
    except Exception:
        return np.nan, np.nan
    return abs(I_exact - I_num), time.perf_counter() - t0

# ======================================================
# DÉFINITION DES TESTS
# ======================================================

tests = [

    # 1. Laguerre
    {
        "nom": "Laguerre",
        "type": "laguerre",
        "I_exact": 3.0,
        "g_lag": lambda x: x**2 + 1,
        "f_classique": lambda x: np.exp(-x)*(x**2 + 1),
        "a": 0,
        "b": 10
    },

    # 2. Tchebychev
    {
        "nom": "Tchebychev",
        "type": "tchebychev",
        "I_exact": np.pi,
        "g_tch": lambda x: np.ones_like(x),
        "f_classique": lambda x: 1/np.sqrt(1-x**2),
        "a": -1,
        "b": 1
    },

    # 3. Combinaison Laguerre + Tchebychev
    {
        "nom": "Combinaison",
        "type": "combinaison",
        "I_exact": float(sp.integrate(sp.exp(-x)/sp.sqrt(1-x**2), (x, 0, 1))),
        "f_classique": lambda x: np.exp(-x)/np.sqrt(1-x**2),
        "a": 0,
        "b": 1
    },

    # 4. Fonction quelconque
    {
        "nom": "Quelconque",
        "type": "quelconque",
        "I_exact": float(sp.integrate(sp.sin(x)+x**2, (x, 0, 2))),
        "f_classique": lambda x: np.sin(x)+x**2,
        "a": 0,
        "b": 2
    }
]

# ======================================================
# BOUCLE PRINCIPALE
# ======================================================

for test in tests:

    erreurs = {m: [] for m in methodes}
    temps   = {m: [] for m in methodes}

    for n in n_vals:
        for m in methodes:

            if test["type"] == "laguerre":

                if m == "Laguerre":
                    e, t = evaluer(lambda n: gauss_laguerre(test["g_lag"], n), n, test["I_exact"])
                else:
                    e, t = evaluer(lambda n: gauss_legendre(test["f_classique"], test["a"], test["b"], n), n, test["I_exact"])

            elif test["type"] == "tchebychev":

                eps = 1e-6
                if m == "Tchebychev":
                    e, t = evaluer(lambda n: gauss_tchebychev(test["g_tch"], n), n, test["I_exact"])
                else:
                    e, t = evaluer(lambda n: gauss_legendre(test["f_classique"], -1+eps, 1-eps, n), n, test["I_exact"])

            else:
                e, t = evaluer(lambda n: gauss_legendre(test["f_classique"], test["a"], test["b"], n), n, test["I_exact"])

            erreurs[m].append(e)
            temps[m].append(t)

    # =============================
    # GRAPHE ERREUR
    # =============================


    eps_plot = 1e-19

    # Copie pour affichage graphique uniquement
    erreurs_plot = {}
    for m in methodes:
        erreurs_plot[m] = [max(e, eps_plot) for e in erreurs[m]]

    fig, (ax_plot, ax_table) = plt.subplots(
        1, 2, figsize=(14, 5),
        gridspec_kw={'width_ratios': [2, 1]}
    )

    # ----- GRAPHE -----
    for m in methodes:
        ax_plot.plot(n_vals, erreurs_plot[m], marker='o', label=m)

    ax_plot.set_yscale("log")
    ax_plot.set_ylim(1e-19, 1e0)
    ax_plot.set_title(f"Erreur numérique — {test['nom']}")
    ax_plot.set_xlabel("n")
    ax_plot.set_ylabel("Erreur")
    ax_plot.grid(True, which="both")
    ax_plot.legend()

    # ----- TABLEAU -----
    ax_table.axis('off')

    table_data = []
    for m in methodes:
        row = [f"{e:.2e}" for e in erreurs[m]]
        table_data.append(row)

    table = ax_table.table(
        cellText=table_data,
        rowLabels=methodes,
        colLabels=[f"n={n}" for n in n_vals],
        loc='center'
    )

    table.scale(1, 1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(9)

    ax_table.set_title("Valeurs des erreurs")

    plt.tight_layout()
    plt.show()

    # =============================
    # GRAPHE TEMPS
    # =============================

    x_pos = np.arange(len(n_vals))
    width = 0.15

    plt.figure()
    for i, m in enumerate(methodes):
        plt.bar(x_pos + i*width, temps[m], width, label=m)

    plt.xticks(x_pos + 2*width, [f"n={n}" for n in n_vals])
    plt.title(f"Temps d'exécution — {test['nom']}")
    plt.ylabel("Temps (s)")
    plt.grid(True, axis="y")
    plt.legend()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# ----------------- Style global plus moderne et lisible -----------------
plt.style.use('ggplot')           # ou 'bmh', 'seaborn-v0_8-whitegrid'
plt.rcParams["figure.figsize"] = (10, 5.5)
plt.rcParams["font.size"] = 11
plt.rcParams["legend.fontsize"] = 9.5
plt.rcParams["axes.formatter.useoffset"] = False

# Liste des couleurs sympa et bien différenciables
COLORS = {
    "Laguerre":   "#d62728",    # rouge
    "Tchebychev": "#1f77b4",    # bleu
    "Legendre":   "#2ca02c",    # vert
    "Simpson":    "#ff7f0e",    # orange
    "Spline":     "#9467bd"     # violet
}

MARKERS = ["o", "s", "^", "D", "v"]

# Dans la boucle principale, après avoir calculé erreurs et temps :

for test in tests:
    #for test in tests:
    erreurs = {m: [] for m in methodes}
    temps   = {m: [] for m in methodes}

    for n in n_vals:
        # 1. On prépare la fonction à intégrer + valeur exacte
        if test["type"] == "laguerre":
            f_to_use = test["f_classique"]      # version exp(-x)*g(x)
            calcul_special = lambda: gauss_laguerre(test["g_lag"], n)
            calcul_classique = lambda: gauss_legendre(f_to_use, 0, 30, n)  # ← 30 ou plus !

        elif test["type"] == "tchebychev":
            f_to_use = test["f_classique"]
            calcul_special = lambda: gauss_tchebychev(test["g_tch"], n)
            calcul_classique = lambda: gauss_legendre(f_to_use, -0.99999, 0.99999, n)

        else:  # combinaison, quelconque...
            f_to_use = test["f_classique"]
            calcul_special = None
            calcul_classique = lambda: gauss_legendre(f_to_use, test["a"], test["b"], n)

        # 2. Évaluation de chaque méthode
        for m in methodes:
            if m == "Laguerre" and test["type"] == "laguerre":
                e, t = evaluer(calcul_special, n, test["I_exact"])
            elif m == "Tchebychev" and test["type"] == "tchebychev":
                e, t = evaluer(calcul_special, n, test["I_exact"])
            elif m in ["Simpson", "Spline"]:
                # Pour ces méthodes on utilise toujours l'intégrale classique
                if m == "Simpson":
                    e, t = evaluer(lambda nn: simpson(f_to_use, test["a"], test["b"], nn), n, test["I_exact"])
                else:
                    e, t = evaluer(lambda nn: spline_quadratique(f_to_use, test["a"], test["b"], nn), n, test["I_exact"])
            else:
                # Legendre et tous les autres cas → gauss-legendre classique
                e, t = evaluer(lambda nn: calcul_classique(), n, test["I_exact"])

            erreurs[m].append(e)
            temps[m].append(t)
    # ───────────────────────────────────────────────
    #     GRAPHIQUE ERREUR - version améliorée
    # ───────────────────────────────────────────────
    fig, (ax, ax_table) = plt.subplots(1, 2, figsize=(15, 6),
                                      gridspec_kw={'width_ratios': [3, 1.1]})

    for i, methode in enumerate(methodes):
        # On masque les valeurs trop petites pour l'affichage log
        err_plot = np.array(erreurs[methode])
        err_plot = np.maximum(err_plot, 1e-17)
        
        ax.semilogy(n_vals, err_plot, 
                   marker=MARKERS[i], 
                   color=COLORS[methode],
                   linewidth=2.1,
                   markersize=8,
                   label=methode)

    ax.set_title(f"Erreur absolue – {test['nom']}", fontsize=13, pad=12)
    ax.set_xlabel("Nombre de points / n", fontsize=11)
    ax.set_ylabel("Erreur absolue |I_exact - I_num|", fontsize=11)
    
    ax.set_yscale("log")
    ax.set_ylim(1e-17, 1)
    ax.grid(True, which="both", ls="--", alpha=0.7)
    
    # Format scientifique propre sur l'axe y
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: f"{y:.0e}"))
    
    ax.legend(loc="upper right", framealpha=0.92, edgecolor="gray")
    
    # Tableau (amélioré)
    ax_table.axis('off')
    table_data = [[f"{e:.2e}" if not np.isnan(e) else "—" for e in erreurs[m]] 
                  for m in methodes]
    
    tab = ax_table.table(cellText=table_data,
                        rowLabels=methodes,
                        colLabels=[f"n = {n}" for n in n_vals],
                        loc='center',
                        cellLoc='center')
    
    tab.scale(1.3, 1.6)
    tab.auto_set_font_size(False)
    tab.set_fontsize(9.5)
    
    for key, cell in tab.get_celld().items():
        cell.set_edgecolor('gray')
        if key[0] == 0:  # ligne d'en-tête
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#f0f0f0')

    fig.suptitle(f"Comparaison des méthodes – {test['nom']}", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

    # ───────────────────────────────────────────────
    #     GRAPHIQUE TEMPS - version améliorée (barres groupées)
    # ───────────────────────────────────────────────
    fig2, ax2 = plt.subplots(figsize=(10, 5.5))
    
    x = np.arange(len(n_vals))
    width = 0.15
    
    for i, methode in enumerate(methodes):
        ax2.bar(x + i*width, temps[methode], width,
                label=methode,
                color=COLORS[methode],
                edgecolor='white', linewidth=0.8)

    ax2.set_title(f"Temps d'exécution – {test['nom']}", fontsize=13, pad=12)
    ax2.set_xlabel("Nombre de points / n")
    ax2.set_ylabel("Temps (secondes)")
    ax2.set_xticks(x + 2*width)
    ax2.set_xticklabels([f"n={n}" for n in n_vals])
    
    ax2.grid(True, axis='y', alpha=0.5, ls="--")
    ax2.legend(loc="upper left", ncol=2, framealpha=0.9)
    
    plt.tight_layout()
    plt.show()